In [ ]:
from soynlp.word import WordExtractor
from soynlp import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
import urllib.request
import re
import csv
import pickle
import numpy as np
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords

In [ ]:
# step1 데이터 업로드
f = pd.read_excel('/content/drive/MyDrive/02)2024-1학기/2_자연어처리/00_개인 프로젝트 논문 (웹툰굿즈)/데이터 및 코드/마루는강쥐_리뷰.xlsx')
df = f

In [ ]:
df["cut_special_character"]=df['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df["cut_korean_consonant"]=df["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df["fiterd_data"]=df["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

In [ ]:
# step2 단어 학습
# 단어 학습 라이브러리 중, LRNounExtractor_v2 선택
# LRNounExtractor_v2 작동 원리 : 보통 한국어 명사 오른쪽에 조사(-가, -를)가 함께 쓰이는 점 고려하여 특정 단어가 명사인지 아닌지 판단
# LRNounExtractor_v2 장점 : 명사 추출 학습기 중 성능 가장 좋음

corpus = df["fiterd_data"]
extractor = LRNounExtractor_v2()
words = extractor.train_extract(corpus)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 948 from 247 sents. mem=0.178 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=1745, mem=0.179 Gb
[Noun Extractor] batch prediction was completed for 330 words
[Noun Extractor] checked compounds. discovered 8 compounds
[Noun Extractor] postprocessing detaching_features : 104 -> 104
[Noun Extractor] postprocessing ignore_features : 104 -> 97
[Noun Extractor] postprocessing ignore_NJ : 97 -> 97
[Noun Extractor] 97 nouns (8 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.180 Gb                    
[Noun Extractor] 32.61 % eojeols are covered


In [ ]:
# step3 토큰화
# 단어 학습 라이브러리 중, LTokenizer 선택
# LTokenizer 작동 원리 : 일반적으로 한국어 어절은 'L(의미) + R(조사)' 구조를 가지므로, 컴퓨터가 계산 시 어절 좌측에 있는 단어에 높은 점수를 부여하여 L(의미)과 R(조사)을 분리

noun = {noun:score[0] for noun, score in words.items() if len(noun) > 2}
l_tokenizer = LTokenizer(noun)
df["Ltokenizer"]=df["fiterd_data"].apply(lambda x: l_tokenizer.tokenize(x))

In [ ]:
keyword = []
for i in df["Ltokenizer"].values:
    for k in i:
        keyword.append(k)
list_drop=keyword
col_names=["keyword"]
df2=pd.DataFrame(list_drop,columns=col_names)

In [ ]:
count = Counter(df2["keyword"])
words_dict = dict(count)

In [ ]:
df3=pd.DataFrame.from_dict({"연관어": words_dict.keys(), "정보량": words_dict.values()}).sort_values("정보량", ascending=False)
df3["연관어"]=df3["연관어"].astype("str")
df3["정보량"]=df3["정보량"].astype("int")
final_list=(df3["연관어"].str.len() >=1)
df4 = df3.loc[final_list]
df4.to_csv('/content/drive/MyDrive/02)2024-1학기/2_자연어처리/00_개인 프로젝트 논문 (웹툰굿즈)/데이터 및 코드/마루는강쥐_선별.csv', encoding='cp949', index=False)